In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt
from sklearn.preprocessing import StandardScaler
#from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

1. read the train dataset into datatrain[]

In [2]:
filepath="C:\\Users\\towermalta\\MLcode\\tcd ml 2019-20 income prediction training (with labels).csv"
datatrain = pd.read_csv(filepath)
datatrain.head()

,Instance,Year of Record,Gender,Age,Country,Size of City,Profession,University Degree,Wears Glasses,Hair Color,Body Height [cm],Income in EUR
0,1,1997.0,0,41.0,Belarus,1239930,steel workers,Bachelor,0,Blond,193,61031.94416
1,2,1996.0,other,41.0,Singapore,1603504,safe event coordinator,Master,0,Black,186,91001.32764
2,3,2018.0,other,28.0,Norway,1298017,receivables/payables analyst,PhD,1,Brown,170,157982.17670
3,4,2006.0,other,33.0,Cuba,751903,fleet assistant,No,1,Black,171,45993.75793
4,5,2010.0,female,46.0,United Arab Emirates,95389,lead trainer,0,0,Blond,188,38022.16217


2. read the test dataset into the datatest[]

In [3]:
filepath="C:\\Users\\towermalta\\MLcode\\tcd ml 2019-20 income prediction test (without labels).csv"
# read_csv里面的参数是csv在你电脑上的路径
datatest = pd.read_csv(filepath)
#读取前五行数据，如果是最后五行，用data.tail()
datatest.head()

,Instance,Year of Record,Gender,Age,Country,Size of City,Profession,University Degree,Wears Glasses,Hair Color,Body Height [cm],Income
0,111994,1992.0,other,21.0,Honduras,391652,senior project analyst,Master,1,Brown,153,NaN
1,111995,1986.0,other,34.0,Kyrgyzstan,33653,greeter,Bachelor,0,Black,163,NaN
2,111996,1994.0,unknown,53.0,Portugal,34765,liaison,Bachelor,1,Blond,153,NaN
3,111997,1984.0,0,29.0,Uruguay,1494132,occupational therapist,No,0,Black,154,NaN
4,111998,2007.0,other,17.0,Serbia,120661,portfolio manager,No,0,Red,191,NaN


3. according to the analysis before, drop the faeture of "wears glasses, hair color", process the feature in both datatrain and datatest

In [4]:
# 选取数据集中有用的特征
datatrain = datatrain.drop(labels=[ 'Wears Glasses', 'Hair Color'], axis=1)
datatrain.head()

,Instance,Year of Record,Gender,Age,Country,Size of City,Profession,University Degree,Body Height [cm],Income in EUR
0,1,1997.0,0,41.0,Belarus,1239930,steel workers,Bachelor,193,61031.94416
1,2,1996.0,other,41.0,Singapore,1603504,safe event coordinator,Master,186,91001.32764
2,3,2018.0,other,28.0,Norway,1298017,receivables/payables analyst,PhD,170,157982.17670
3,4,2006.0,other,33.0,Cuba,751903,fleet assistant,No,171,45993.75793
4,5,2010.0,female,46.0,United Arab Emirates,95389,lead trainer,0,188,38022.16217


In [5]:
# 选取数据集中有用的特征
datatest = datatest.drop(labels=[ 'Wears Glasses', 'Hair Color'], axis=1)
datatest.head()

,Instance,Year of Record,Gender,Age,Country,Size of City,Profession,University Degree,Body Height [cm],Income
0,111994,1992.0,other,21.0,Honduras,391652,senior project analyst,Master,153,NaN
1,111995,1986.0,other,34.0,Kyrgyzstan,33653,greeter,Bachelor,163,NaN
2,111996,1994.0,unknown,53.0,Portugal,34765,liaison,Bachelor,153,NaN
3,111997,1984.0,0,29.0,Uruguay,1494132,occupational therapist,No,154,NaN
4,111998,2007.0,other,17.0,Serbia,120661,portfolio manager,No,191,NaN


4. divide the data(datatrain and datatest) into two parts
   first part:"gender, country, profession, university degree"
   second part:"year of record, age, body height, size of city"

In [6]:
data_object1 = pd.DataFrame(datatrain, columns=['Gender', 'Country', 'Profession', 'University Degree'], index=datatrain.index)
data_num1 = pd.DataFrame(datatrain, columns=['Year of Record', 'Age', 'Body Height [cm]','Size of City'], index=datatrain.index)

data_object2 = pd.DataFrame(datatest, columns=['Gender', 'Country', 'Profession', 'University Degree'], index=datatest.index)
data_num2 = pd.DataFrame(datatest, columns=['Year of Record', 'Age', 'Body Height [cm]','Size of City'], index=datatest.index)


5. to fill the missing value with most frequent value
   use simpleimputer() in sklearn

In [7]:
#填充有缺失值的行
from sklearn.impute import SimpleImputer
#SimpleImputer中输入的至少是二维矩阵  
simple = SimpleImputer(missing_values = np.nan,strategy="most_frequent")
data_object1 = simple.fit_transform(data_object1.values)
data_num1 = simple.fit_transform(data_num1.values)

data_object2 = simple.fit_transform(data_object2.values)
data_num2 = simple.fit_transform(data_num2.values)

6. use ordinalEncoder() to encode the object value

In [8]:
#对object类型的数据进行编码
from sklearn import preprocessing
encoder=preprocessing.OrdinalEncoder()
data_object1=encoder.fit_transform(data_object1)
X = np.c_[data_num1,data_object1]

print(X)

[[1.997e+03 4.100e+01 1.930e+02 ... 1.200e+01 1.209e+03 1.000e+00]
 [1.996e+03 4.100e+01 1.860e+02 ... 1.230e+02 1.048e+03 2.000e+00]
 [2.018e+03 2.800e+01 1.700e+02 ... 1.060e+02 1.008e+03 4.000e+00]
 ...
 [1.993e+03 3.600e+01 1.530e+02 ... 1.320e+02 7.890e+02 2.000e+00]
 [2.019e+03 5.400e+01 1.900e+02 ... 4.100e+01 1.085e+03 3.000e+00]
 [2.017e+03 2.700e+01 1.740e+02 ... 1.330e+02 9.620e+02 3.000e+00]]


7. got the value of target

In [9]:
#获得y值
train_income = pd.DataFrame(datatrain, columns=['Income in EUR'], index=datatrain.index)
#y = data_conti.values
y = train_income.values
print(y)

[[ 61031.94416]
 [ 91001.32764]
 [157982.1767 ]
 ...
 [289951.3294 ]
 [100046.5278 ]
 [145886.2885 ]]


8. divide the train dataset into X_train, X_test, y_train, y_test

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print (X_train.shape)
print (y_train.shape)
print (X_test.shape)
print (y_test.shape)

(89594, 8)
(89594, 1)
(22399, 8)
(22399, 1)


9. use minmaxscaler() to scale the feature of X_train

In [11]:
#对特征进行缩放
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
#特征很多的时候使用MinMaxScaler().partial_fit(data)来代替fit否则会报错
#scaler.fit(X)  #在这里本质是生成min(x)和max(x)
X_train = scaler.fit_transform(X_train)  #通过接口导出结果
print(X_train)

[[0.94871795 0.18811881 0.59064327 ... 0.95597484 0.69678865 0.25      ]
 [0.94871795 0.0990099  0.59649123 ... 0.83018868 0.82374907 0.75      ]
 [0.02564103 0.41584158 0.64912281 ... 0.89937107 0.7356236  0.5       ]
 ...
 [0.41025641 0.21782178 0.28654971 ... 0.50314465 0.52352502 0.5       ]
 [0.1025641  0.16831683 0.48538012 ... 0.7672956  0.5690814  0.75      ]
 [0.92307692 0.01980198 0.39766082 ... 0.28301887 0.86407767 0.25      ]]


10. use LinearRegression to train the train dataset

In [12]:
#训练模型
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, y_train)

#from sklearn.ensemble import RandomForestRegressor
#forest_reg = RandomForestRegressor()
#forest_reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

11. encode the X_test

In [13]:
data_object2=encoder.fit_transform(data_object2)
X_test1 = np.c_[data_num2,data_object2]
print(X_test1)

[[1.992e+03 2.100e+01 1.530e+02 ... 6.000e+01 1.115e+03 2.000e+00]
 [1.986e+03 3.400e+01 1.630e+02 ... 7.200e+01 5.780e+02 1.000e+00]
 [1.994e+03 5.300e+01 1.530e+02 ... 1.110e+02 7.150e+02 1.000e+00]
 ...
 [2.019e+03 5.000e+01 1.620e+02 ... 1.350e+02 7.890e+02 2.000e+00]
 [1.980e+03 5.400e+01 1.620e+02 ... 9.800e+01 6.730e+02 2.000e+00]
 [2.017e+03 4.100e+01 1.970e+02 ... 1.430e+02 7.560e+02 2.000e+00]]


12. combine the dataset of test before with the new X_test
    this is bigger than the original X_test
    but will perform better

In [14]:
X_test2 = np.concatenate((X_test,X_test1),axis=0)
print(X_test2.shape)
print(X_test2)

(95629, 8)
[[2.011e+03 2.400e+01 1.720e+02 ... 1.360e+02 1.072e+03 1.000e+00]
 [1.995e+03 2.300e+01 1.780e+02 ... 7.300e+01 7.000e+01 1.000e+00]
 [1.995e+03 4.800e+01 2.100e+02 ... 1.130e+02 9.740e+02 1.000e+00]
 ...
 [2.019e+03 5.000e+01 1.620e+02 ... 1.350e+02 7.890e+02 2.000e+00]
 [1.980e+03 5.400e+01 1.620e+02 ... 9.800e+01 6.730e+02 2.000e+00]
 [2.017e+03 4.100e+01 1.970e+02 ... 1.430e+02 7.560e+02 2.000e+00]]


13. scale the testX dataset

In [15]:
X_test = scaler.fit_transform(X_test2)  #通过接口导出结果
print(X_test)

[[0.79487179 0.08928571 0.45637584 ... 0.85534591 0.80059746 0.25      ]
 [0.38461538 0.08035714 0.4966443  ... 0.4591195  0.05227782 0.25      ]
 [0.38461538 0.30357143 0.7114094  ... 0.71069182 0.72740851 0.25      ]
 ...
 [1.         0.32142857 0.38926174 ... 0.8490566  0.58924571 0.5       ]
 [0.         0.35714286 0.38926174 ... 0.6163522  0.50261389 0.5       ]
 [0.94871795 0.24107143 0.62416107 ... 0.89937107 0.56460045 0.5       ]]


14. use the testX to predict the income 

In [16]:
#y_pred = forest_reg.predict(X_test2)
y_pred = linreg.predict(X_test)
print(y_pred)

[[108353.0345478 ]
 [ 16930.05210477]
 [134827.46341319]
 ...
 [140670.48634148]
 [ 61700.29470528]
 [140642.4863915 ]]


In [18]:
income = pd.DataFrame(y_pred, columns=['Income'])
print(income)

              Income
0      108353.034548
1       16930.052105
2      134827.463413
3      147523.209325
4      109886.356222
...              ...
95624  140883.081288
95625  149524.636216
95626  140670.486341
95627   61700.294705
95628  140642.486391

[95629 rows x 1 columns]


15. put the result into a file

In [19]:
income.to_csv('linearimpro_pred.csv') 

# tree model

#训练模型
from sklearn.tree import DecisionTreeRegressor 
tree_reg = DecisionTreeRegressor()
tree_reg.fit(X_train,y_train)

y_pred = tree_reg.predict(X_test)
print(y_pred)

# random forest model

from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(X_train, y_train)

y_pred = forest_reg.predict(X_test)
print(y_pred)